# Tools and Agents in Strands

**Building Intelligent AI Agents with Custom Tools**

---

Welcome to this comprehensive tutorial on **tools and agents** in the Strands framework! This notebook provides a hands-on introduction to creating AI agents that can perform complex tasks by integrating with external tools and services. By the end of this 10-minute tutorial, you'll understand how to build powerful, tool-equipped agents.

### 🎯 What You'll Learn

In this tutorial, you will:
- Create agents with custom behaviors and personalities
- Build custom tools using the `@tool` decorator
- Understand type hints and their importance for tools
- Implement conversation context and memory
- Monitor agent performance and handle errors
- Create production-ready tool-equipped agents

### 🛠️ What Are Tools?

**Tools** in Strands are functions that agents can use to:
- Perform calculations or data processing
- Access external APIs and services
- Interact with databases or file systems
- Execute any custom logic you define

Think of tools as superpowers you give to your agents!

## 📦 Step 1: Installing Required Packages

### Overview
Let's start by installing the necessary packages for this tutorial.

### 📚 Packages We'll Install
- **strands-agents**: Core framework for building agents
- **strands-agents-tools**: Additional tool utilities
- **strands-agents-builder**: Agent configuration helpers

In [ ]:
# Install Strands Agents SDK and related packages
%pip install strands-agents strands-agents-tools strands-agents-builder -q

print("✅ All packages installed successfully!")
print("   Ready to build intelligent agents with tools! 🚀")

## 🔐 Step 2: Setting Up AWS Authentication

### Overview
Strands Agents SDK supports multiple model providers. We'll use AWS Bedrock to access Claude, one of the most powerful language models available.

### 🔑 Authentication Options
1. **AWS Profile** (Recommended for development)
2. **Environment Variables**
3. **Direct Credentials** (Less secure)
4. **IAM Roles** (Recommended for production)

### 📋 Required Permissions
Ensure your AWS credentials have:
- `bedrock:InvokeModel`
- `bedrock:InvokeModelWithResponseStream`

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Configure AWS session
session = boto3.Session(
    # aws_access_key_id='your_access_key',
    # aws_secret_access_key='your_secret_key',
    # aws_session_token='your_session_token',  # If using temporary credentials
    # region_name='us-west-2',
    profile_name='default'  # Optional: Use a specific AWS profile
)

# Create a Bedrock model instance
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    boto_session=session
)

print("✅ AWS Bedrock configured successfully!")
print(f"   Model: Claude 3.7 Sonnet")
print(f"   Profile: {session.profile_name}")

## 🤖 Step 3: Creating a Basic Agent

### Starting Simple
Let's create our first agent without any tools. This demonstrates the basic agent capabilities before we add custom functionality.

In [ ]:
# Create a basic agent
basic_agent = Agent(model=bedrock_model)

print("🤖 Basic agent created!")
print("   Type: Standard Agent")
print("   Tools: None (yet!)")

# Test the basic agent
response = basic_agent("What is artificial intelligence?")
print("\n💬 Agent's response:")
print(response)

## 🎨 Step 4: Creating Agents with Custom Personalities

### System Prompts
System prompts define an agent's personality, expertise, and behavior. Let's create specialized agents for different purposes.

In [ ]:
# Create a Python expert agent
python_expert = Agent(
    system_prompt="""You are a senior Python developer with 15 years of experience.
    You specialize in clean code, best practices, and teaching others.
    Always provide clear explanations and practical examples.""",
    model=bedrock_model,
)

# Create a creative writing assistant
creative_writer = Agent(
    system_prompt="""You are a creative writing assistant with a vivid imagination.
    You help users craft engaging stories, develop characters, and improve their writing.
    Be encouraging and provide constructive feedback.""",
    model=bedrock_model,
)

print("🎨 Specialized agents created!")
print("   1. Python Expert - For coding questions")
print("   2. Creative Writer - For storytelling")

# Test the Python expert
question = "How do I create a list comprehension in Python?"
response = python_expert(question)
print(f"\n💻 Python Expert's response to '{question}':")
print(response)

## 🔧 Step 5: Building Custom Tools

### The Power of Tools
Tools transform agents from conversational AI into action-oriented assistants. Let's create some useful tools!

### 🎯 Key Concepts
1. **@tool Decorator**: Marks a function as a tool
2. **Type Hints**: Critical for proper parameter conversion
3. **Docstrings**: Help the agent understand when to use each tool

In [ ]:
from strands import tool
import datetime
import random
import math

# Tool 1: Get current time
@tool
def get_current_time():
    """Get the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Tool 2: Generate random numbers
@tool
def generate_random_number(min_val: int = 1, max_val: int = 100):
    """Generate a random number between min_val and max_val.
    
    Args:
        min_val: Minimum value (default: 1)
        max_val: Maximum value (default: 100)
    """
    return random.randint(min_val, max_val)

# Tool 3: Calculate factorial
@tool
def calculate_factorial(n: int):
    """Calculate the factorial of a number.
    
    Args:
        n: The number to calculate factorial for
    """
    if n < 0:
        return "Error: Factorial is not defined for negative numbers"
    elif n == 0 or n == 1:
        return 1
    else:
        return math.factorial(n)

# Tool 4: Calculate compound interest
@tool
def calculate_compound_interest(principal: float, rate: float, time: int, compounds_per_year: int = 12):
    """Calculate compound interest.
    
    Args:
        principal: Initial amount
        rate: Annual interest rate (as a percentage)
        time: Time period in years
        compounds_per_year: Number of times interest compounds per year (default: 12)
    """
    rate_decimal = rate / 100
    amount = principal * (1 + rate_decimal/compounds_per_year) ** (compounds_per_year * time)
    interest = amount - principal
    return {
        "principal": principal,
        "final_amount": round(amount, 2),
        "interest_earned": round(interest, 2),
        "rate": rate,
        "time": time
    }

print("🔧 Custom tools created!")
print("   1. get_current_time - Returns current date/time")
print("   2. generate_random_number - Random number generator")
print("   3. calculate_factorial - Mathematical calculations")
print("   4. calculate_compound_interest - Financial calculations")

## 🤖 Step 6: Creating a Tool-Equipped Agent

### Bringing It All Together
Now let's create an agent that can use all our custom tools. This agent will be able to perform calculations, generate random numbers, and more!

In [ ]:
# Create a tool-equipped agent
tool_agent = Agent(
    system_prompt="""You are a helpful assistant equipped with various tools.
    
    🛠️ YOUR CAPABILITIES:
    - Time and date information
    - Random number generation
    - Mathematical calculations (factorial)
    - Financial calculations (compound interest)
    
    📋 GUIDELINES:
    - Use tools when appropriate to provide accurate information
    - Explain your calculations and results clearly
    - If a user's request is unclear, ask for clarification
    - Be friendly and helpful in all interactions""",
    tools=[
        get_current_time,
        generate_random_number,
        calculate_factorial,
        calculate_compound_interest
    ],
    model=bedrock_model,
)

print("🤖 Tool-equipped agent created!")
print("   Personality: Helpful assistant")
print("   Tools: 4 custom functions")
print("   Ready to help with calculations and more!")

## 🎯 Step 7: Testing Our Tool-Equipped Agent

### Real-World Examples
Let's test our agent with various tasks that require tool usage. Watch how it intelligently decides which tools to use!

In [ ]:
# Test various tool capabilities
test_questions = [
    "What time is it right now?",
    "Generate a random number between 1 and 50",
    "What's the factorial of 7?",
    "If I invest $10,000 at 5% annual interest for 10 years, how much will I have?",
    "Generate 3 random numbers between 100 and 200"
]

print("🎯 Testing tool-equipped agent...\n")

for i, question in enumerate(test_questions, 1):
    print(f"{'='*60}")
    print(f"Question {i}: {question}")
    print(f"{'='*60}")
    
    response = tool_agent(question)
    print(f"🤖 Agent's response:")
    print(response)
    print()

## 💬 Step 8: Demonstrating Conversation Context

### Memory and Context
Strands agents maintain conversation history, allowing for natural, contextual interactions. Let's see this in action!

In [ ]:
# Create a fresh agent for conversation demo
conversation_agent = Agent(
    system_prompt="You are a friendly math tutor. Help students understand concepts step by step.",
    tools=[calculate_factorial, generate_random_number],
    model=bedrock_model
)

print("💬 Starting a contextual conversation...\n")

# Conversation flow
conversation = [
    "Hi! I'm learning about factorials.",
    "Can you calculate 5 factorial for me?",
    "How is that different from 6 factorial?",
    "Can you show me a pattern with factorials from 1 to 5?"
]

for turn, message in enumerate(conversation, 1):
    print(f"\n{'='*60}")
    print(f"Turn {turn} - You: {message}")
    print(f"{'='*60}")
    
    response = conversation_agent(message)
    print(f"🤖 Tutor: {response}")

print("\n✨ Notice how the agent maintains context throughout the conversation!")

## ⚠️ Step 9: Error Handling and Edge Cases

### Building Robust Agents
Good agents handle errors gracefully. Let's test how our agent deals with invalid inputs and edge cases.

In [ ]:
# Test error handling
error_test_cases = [
    "Calculate the factorial of -5",
    "Generate a random number between 100 and 50",  # Min > Max
    "What's the compound interest on negative $1000?",
    ""  # Empty input
]

print("⚠️ Testing error handling...\n")

for i, test_case in enumerate(error_test_cases, 1):
    print(f"Test {i}: '{test_case}'")
    try:
        response = tool_agent(test_case) if test_case else tool_agent(" ")
        print(f"✅ Response: {response}")
    except Exception as e:
        print(f"❌ Error caught: {e}")
    print("-" * 40)

print("\n✅ Error handling test completed!")

## 📊 Step 10: Performance Monitoring

### Understanding Agent Performance
Let's analyze how our agents perform and understand the overhead of tool usage.

In [ ]:
import time

def measure_performance(agent, question, description):
    """Measure agent response time and analyze performance"""
    print(f"\n📊 Testing: {description}")
    print(f"   Question: {question}")
    
    start_time = time.time()
    response = agent(question)
    end_time = time.time()
    
    response_time = end_time - start_time
    word_count = len(str(response).split())
    
    print(f"   ⏱️  Response time: {response_time:.2f} seconds")
    print(f"   📝 Response length: {word_count} words")
    print(f"   ⚡ Words per second: {word_count/response_time:.2f}")
    
    return response_time

# Compare performance
print("📊 PERFORMANCE COMPARISON")
print("=" * 60)

# Test without tools
time1 = measure_performance(
    basic_agent,
    "Explain what a factorial is",
    "Basic agent (no tools)"
)

# Test with tools
time2 = measure_performance(
    tool_agent,
    "Calculate the factorial of 8 and explain what it means",
    "Tool-equipped agent"
)

print(f"\n📈 Performance Analysis:")
print(f"   Tool overhead: {time2 - time1:.2f} seconds")
print(f"   Overhead percentage: {((time2 - time1) / time1 * 100):.1f}%")

## 🎓 Advanced: Creating a Multi-Tool Research Assistant

### Combining Multiple Capabilities
Let's create a sophisticated research assistant that combines multiple tools for complex tasks.

In [ ]:
# Additional tools for research
@tool
def word_count(text: str):
    """Count the number of words in a text."""
    return len(text.split())

@tool
def calculate_reading_time(word_count: int, words_per_minute: int = 200):
    """Calculate estimated reading time in minutes.
    
    Args:
        word_count: Number of words in the text
        words_per_minute: Reading speed (default: 200)
    """
    minutes = word_count / words_per_minute
    return f"{minutes:.1f} minutes"

# Create research assistant
research_assistant = Agent(
    system_prompt="""You are an advanced research assistant capable of complex analysis.
    
    Your capabilities include:
    - Mathematical calculations
    - Text analysis
    - Time estimations
    - Data synthesis
    
    Always provide comprehensive, well-structured responses.""",
    tools=[
        calculate_factorial,
        calculate_compound_interest,
        word_count,
        calculate_reading_time,
        get_current_time,
        generate_random_number
    ],
    model=bedrock_model
)

# Complex research task
research_task = """I'm writing a 5000-word research paper about compound interest. 
Can you:
1. Calculate how long it will take to read
2. Show me how $1000 grows over 20 years at 7% interest
3. Give me 5 random page numbers between 1-50 for citations
"""

print("🎓 Testing Research Assistant")
print("=" * 80)
print(f"Task: {research_task}")
print("=" * 80)

response = research_assistant(research_task)
print("\n📚 Research Assistant's Response:")
print(response)

## 🎉 Congratulations!

### 🏆 What You've Accomplished
In this tutorial, you've learned how to:
- ✅ Create basic and specialized agents
- ✅ Build custom tools with the `@tool` decorator
- ✅ Use type hints for reliable tool execution
- ✅ Implement conversation context and memory
- ✅ Handle errors gracefully
- ✅ Monitor and optimize performance
- ✅ Combine multiple tools for complex tasks

### 🚀 Key Takeaways

1. **Tools = Superpowers**: Tools transform conversational agents into action-oriented assistants
2. **Type Hints Matter**: Proper type hints ensure reliable tool execution
3. **Context is King**: Agents maintain conversation history for natural interactions
4. **Error Handling**: Robust agents handle edge cases gracefully

### 💡 Best Practices

1. **Clear Docstrings**: Help agents understand when to use each tool
2. **Type Annotations**: Always include type hints for parameters
3. **Error Handling**: Handle edge cases within your tools
4. **System Prompts**: Design clear instructions for agent behavior

### 🔮 What's Next?

Now that you've mastered tools and agents, you're ready to:
1. **Build Complex Tools**: Integrate with APIs, databases, and external services
2. **Create Specialized Agents**: Domain-specific assistants for your needs
3. **Implement Workflows**: Chain multiple agents and tools together
4. **Deploy to Production**: Build real-world applications

### 📚 Resources

- [Strands Documentation](https://strandsagents.com/0.1.x/)
- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Tool Development Guide](https://strandsagents.com/0.1.x/user-guide/concepts/tools/)

### 🌟 Challenge Yourself

Try creating:
- A weather tool that fetches current conditions
- A database query tool for data analysis
- An email sending tool for notifications
- A file processing tool for document analysis

The possibilities are endless with Strands Agents!

Happy building! 🚀🤖✨